In [1]:
!pip install transformers
!pip install datasets
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 3.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.9 MB/s eta 0:00:0

In [2]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from datasets import Dataset
from google.colab import drive
from sklearn.metrics import mean_squared_error
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

drive.mount('/content/drive')
project_path = './drive/MyDrive/Colab Notebooks/GPT_community/'


def compute_metrics_for_regression(eval_pred):
    """Define this metric specifically to check model is correctly
    using mse as loss function."""
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)

    return {'mse':mse}

Mounted at /content/drive


In [3]:
# Load CSV containing scalar rewards for GPT2 responses to philosophy prompts
rewards_file = os.path.join(project_path, 'data/rewards.csv')
df_rewards = pd.read_csv(rewards_file).rename(columns={'response':'text', 'reward':'labels'})[['text', 'labels']]
display(df_rewards.tail())

,text,labels
97,Is religion a force for good?\n\nThe answer is...,0.5
98,"Is religion a force for good? I think so.""\n\n...",0.0
99,Are we biological machines? We're just our own...,1.0
100,Are we biological machines?\n\nYes we are.\n\n...,0.5
101,"Are we biological machines?\n\nWell, we're all...",0.0


In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Create dataset object and split into train and test sets
dataset = Dataset.from_pandas(df_rewards)
dataset = dataset.train_test_split(test_size=0.2)

# Define a function to tokenize the text
def tokenize_function(example):
  encoding = tokenizer(example['text'], padding='max_length', truncation=True)
  encoding['labels'] = torch.tensor(example['labels']).unsqueeze(1)
  return encoding

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 81
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21
    })
})

In [5]:
# Load BERT model for sequence classification and set num_labels=1 to make it do regression
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 1).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
# Define relevant parameters and train

batch_size = 8

args = TrainingArguments(
    output_dir='trained_model',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_for_regression
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 81
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 55
  Number of trainable parameters = 109483009
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to e

Epoch,Training Loss,Validation Loss,Mse
1,No log,0.241635,0.241635
2,No log,0.218713,0.218713
3,No log,0.216019,0.216019
4,No log,0.234433,0.234433
5,No log,0.239335,0.239335


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 21
  Batch size = 8
Saving model checkpoint to trained_model/checkpoint-11
Configuration saved in trained_model/checkpoint-11/config.json
Model weights saved in trained_model/checkpoint-11/pytorch_model.bin
tokenizer config file saved in trained_model/checkpoint-11/tokenizer_config.json
Special tokens file saved in trained_model/checkpoint-11/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 21
  Batch size = 8

TrainOutput(global_step=55, training_loss=0.14665367820046166, metrics={'train_runtime': 69.4642, 'train_samples_per_second': 5.83, 'train_steps_per_second': 0.792, 'total_flos': 106559020661760.0, 'train_loss': 0.14665367820046166, 'epoch': 5.0})

In [10]:
# Test some sentences to see if the reward model is doing something sensible
# test_sentence = "What is the good life? Probably eating crisps and watching television. Then going to the pub and drinking beer by myself." # reward = 0.39
test_sentence = "What is the good life? To examine the good life we must take a number of variables into account. First, why is anything 'good'?" # reward = 0.62
input_ids = tokenizer.encode(test_sentence, return_tensors='pt').to('cuda')
outputs = model(input_ids).logits.item()
outputs

0.6215947270393372

In [11]:
# Push model to Hugging Face
model.push_to_hub('rjuggins/philosophy_reward_test', use_auth_token='')

Configuration saved in /tmp/tmpnnvk6pcv/config.json
Model weights saved in /tmp/tmpnnvk6pcv/pytorch_model.bin
Uploading the following files to rjuggins/philosophy_reward_test: pytorch_model.bin,config.json


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rjuggins/philosophy_reward_test/commit/c4f57f200ffeba2cad5ccead6ac82cc599ed8cfc', commit_message='Upload BertForSequenceClassification', commit_description='', oid='c4f57f200ffeba2cad5ccead6ac82cc599ed8cfc', pr_url=None, pr_revision=None, pr_num=None)